<a href="https://colab.research.google.com/github/hexabenja/Scr4perSodimac/blob/main/DataWebScr4per.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Nueva sección


Esta celda instala las bibliotecas necesarias para el web scraping con Langchain, incluidas langchain-community y beautifulsoup4.

In [1]:
%pip install -qU langchain-community beautifulsoup4
%pip install -qU langchain-unstructured

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 35.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


Esta celda importa bs4 y WebBaseLoader para cargar contenido de una página web específica (En este ejemplo catálogo CREST de sodimac).

In [16]:
from langchain_community.document_loaders import WebBaseLoader
import bs4
import re
import csv
from datetime import datetime
import time
import asyncio

 Inicializa el loader con la URL y obtiene el documento de forma asíncrona.

Agrega imagenes a parte de los datos

In [3]:
def get_product_images_from_soup(soup):
    """Extrae URLs de imágenes desde el soup"""
    images = soup.find_all('img', class_='jsx-1996933093')
    image_urls = []
    for img in images:
        src = img.get('src', '')
        if src:
            image_urls.append(src)
    return image_urls

In [17]:
page_url = "https://www.sodimac.cl/sodimac-cl/marca/WEBER"

loader = WebBaseLoader(web_paths=[page_url])
docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs)
await asyncio.sleep(5) # Evitar que detecte como spam el fetching asincronico (sesion debe ser mayor a 1 segundo)

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  4.27it/s]


Esta celda inicializa WebBaseLoader nuevamente, pero esta vez con parámetros adicionales para `bs_kwargs` y `bs_get_text_kwargs` para analizar específicamente solo el contenido de Markdown dentro de la clase `theme-doc-markdown` y para usar separadores y eliminaciones personalizados.

In [18]:
loader = WebBaseLoader(
    web_paths=[page_url],
    bs_kwargs={
        "parse_only": bs4.SoupStrainer(id="testId-searchResults-products", class_="jsx-636341914 search-results--products"),
    },
    bs_get_text_kwargs={"separator": ",", "strip": True},
)

soup = loader.scrape()
image_urls = get_product_images_from_soup(soup)

docs = []
async for doc in loader.alazy_load():
    docs.append(doc)

assert len(docs) == 1
doc = docs[0]

Fetching pages: 100%|##########| 1/1 [00:00<00:00,  5.55it/s]


Esta celda elimina ciertas frases del panel de los resultados con una definición. Al detectar frases tales mencionadas abajo las va a detectar como definición unwanted_phrases.

Limpia el contenido definiendolo como cleaned_content. Usa la definición de cleaned_content para reemplazar por "". Las frases unwanted_phrases de celda superior. Elimina números entre parentesís de puntuacion disponible. Elimina residuos de texto.

In [19]:
#Sintaxis de expresiones regulares regex:
# r'...' = es un raw String por lo cual es texto
# ( ) = es inicio y fin de captura de caracteres
# Backslash es un escape de la funcion especial y debe agregarse como un valor string el valor siguiente (en este caso $)
# \s* = es Cero o mas espacios en blanco
# [ ] = Inicio y final de conjunto de caracteres
# \d = Digito del 1 al 9
# + = Mas conjunto anterior
# ,\d+, = Coma mas uno o mas digitos mas coma
# {2,} = Dos o mas repeticiones de la coma
# ^ = Inicio de lina
# | = Disyuncion OR
# $ = Final de linea
#

def parse_products_with_images(content, image_urls):
    # Lista de frases a eliminar
    unwanted_phrases = [
        "cuotas sin interés",
        "Agregar al Carro",
        "Por Sodimac",
        "Llega mañana",

    ]

    # Limpiar el contenido
    cleaned_content = content

    # Eliminar frases no deseadas
    for phrase in unwanted_phrases:
        cleaned_content = cleaned_content.replace(phrase, "")

    # Eliminar el numero de cuotas)
    cleaned_content = re.sub(r'(\$\s*[\d,\.]+),\d+,', r'\1,', cleaned_content)

    # Eliminar números entre paréntesis como (78), (13), etc y reemplazarlos por nueva fila/linea
    cleaned_content = re.sub(r',?\s*\(\d+\),?', '|NEWLINE|', cleaned_content)

    # Limpiar comas múltiples y espacios extra
    cleaned_content = re.sub(r',{2,}', ',', cleaned_content)
    cleaned_content = re.sub(r',\s*,', ',', cleaned_content)
    cleaned_content = re.sub(r'^\s*,+|,+\s*$', '', cleaned_content)

    # Separar productos por "newline"
    products = []
    parts = cleaned_content.split("|NEWLINE|")

    # Agregar tag a URL de Google Lens a IMG Sodimac
    img_index = 0
    for part in parts:
        part = part.strip().strip(',').strip()
        if part:
            if img_index < len(image_urls):
                url_imagen = image_urls[img_index]
                google_lens_url = f"https://lens.google.com/uploadbyurl?url={url_imagen}&q=SITE%3ASODIMAC.CL"
                product_line = f"{part},{google_lens_url}"
                img_index += 1
            else:
                product_line = f"{part},Sin imagen"
            products.append(product_line)

    return products

CELDA PARA COMPROBAR SI EL TEXTO PRINTEA CORRECTAMENTE

In [20]:
print(f"{doc.metadata}\n")
print(doc.page_content[:1000].strip())

{'source': 'https://www.sodimac.cl/sodimac-cl/marca/WEBER'}

WEBER,Adhesivo Cerámico Muro Superficie Flexible 25 kg,Por Sodimac,$  20.830,6,cuotas sin interés,(101),Agregar al Carro,WEBER,Fragüe Grey 1 kg,Por Sodimac,$  2.490,6,cuotas sin interés,(15),Agregar al Carro,WEBER,Parrilla a Carbón Original Kettle 22" Acero,Por Sodimac,$  129.990,-35%,$  199.990,(1390),Agregar al Carro,WEBER,Adhesivo Cerámico Piso y muro Superficie Rígido 25 kg,Por Sodimac,$  5.090,$  4.937,6,cuotas sin interés,(40),Agregar al Carro,WEBER,Adhesivo Porcelanato Piso y muro Superficie Rígido 25 kg,Por Sodimac,$  9.280,$  9.002,6,cuotas sin interés,(271),Agregar al Carro,WEBER,6 kilos Masilla para junturas de fibrocemento en pasta.,Por Sodimac,$  11.990,(48),Agregar al Carro,WEBER,Fragüe Almond 1 kg,Por Sodimac,$  2.490,6,cuotas sin interés,(2),Agregar al Carro,WEBER,Adhesivo cerámico polvo AC 20 kg,Por Imperial,$  14.990,Agregar al Carro,WEBER,Autonivelante Superficie Rígido 25 kg,Por Sodimac,$  17.990,6,cuotas 

Celda para parsear el texto final delimitado por comas

In [23]:
raw_content = doc.page_content
clean_products = parse_products_with_images(raw_content, image_urls)

final_text = "\n".join(clean_products)
print(final_text)

WEBER,Adhesivo Cerámico Muro Superficie Flexible 25 kg,$  20.830,https://lens.google.com/uploadbyurl?url=https://media.sodimac.cl/sodimacCL/5437970_001/width=240,height=240,quality=70,format=webp,fit=pad&q=SITE%3ASODIMAC.CL
WEBER,Fragüe Grey 1 kg,$  2.490,https://lens.google.com/uploadbyurl?url=https://media.sodimac.cl/sodimacCL/4203666_001/width=240,height=240,quality=70,format=webp,fit=pad&q=SITE%3ASODIMAC.CL
WEBER,Parrilla a Carbón Original Kettle 22" Acero,$  129.990,-35%,$  199.990,https://lens.google.com/uploadbyurl?url=https://media.sodimac.cl/sodimacCL/5346630_01/width=240,height=240,quality=70,format=webp,fit=pad&q=SITE%3ASODIMAC.CL
WEBER,Adhesivo Cerámico Piso y muro Superficie Rígido 25 kg,$  5.090,$  4.937,https://lens.google.com/uploadbyurl?url=https://media.sodimac.cl/sodimacCL/3334171_001/width=240,height=240,quality=70,format=webp,fit=pad&q=SITE%3ASODIMAC.CL
WEBER,Adhesivo Porcelanato Piso y muro Superficie Rígido 25 kg,$  9.280,$  9.002,https://lens.google.com/uploadby

In [26]:
# Función simple para exportar a CSV con encabezados
def export_simple_csv(clean_products):
    if clean_products:
        first_product = clean_products[0]
        marca = first_product.split(',')[0].strip()
    else:
        marca = "productos"

    fecha_actual = datetime.now().strftime("%Y%m%d")
    filename = f"{marca}_{fecha_actual}.csv"

    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.writer(csvfile)

        # Agregar columnas
        writer.writerow(['URL_Google_Lens', 'Marca', 'Nombre_Producto', 'Precio', 'Ofertas'])

        for product in clean_products:
            parts = product.split(',')
            url_google_lens = parts[0].strip()
            marca = parts[1].strip() if len(parts) > 1 else ''
            nombre_producto = parts[2].strip() if len(parts) > 2 else ''
            precio = parts[3].strip() if len(parts) > 3 else ''
            ofertas = parts[3].strip() if len(parts) > 4 else ''

            writer.writerow([marca, nombre_producto, precio, url_google_lens, ofertas])  # MODIFICADO

    print(f"CSV creado: {filename}")
# Usar la función:
export_simple_csv(clean_products)

CSV creado: WEBER_20251016.csv
